In [35]:
import os
import shutil
import sys

In [3]:
%pwd

'd:\\Artificial Intelligence and Machine Learning\\End2End Projects\\chicken-disease-classification\\research'

In [4]:
os.chdir("../")

In [38]:
%pwd

'd:\\Artificial Intelligence and Machine Learning\\End2End Projects\\chicken-disease-classification'

In [39]:
## Define Entity for Data Ingestion
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir: Path
    dataset_name: str
    local_data_file: Path
    unzip_dir: Path

In [40]:
from src.cnnclassifier.constants import *
from cnnclassifier.utils.common import *

In [41]:
## Update Configuration Manager
class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH, 
                 params_filepath = PARAMS_FILE_PATH,
                 secret_filepath = SECRET_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.secret = read_yaml(secret_filepath)

        create_directories_files([self.config.artifacts_root])

    
    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories_files([config.root_dir])

        data_ingestion_config = DataIngestionConfig(root_dir=config.root_dir,
                                                    dataset_name = config.dataset_name,
                                                    local_data_file = config.local_data_file,
                                                    unzip_dir = config.unzip_dir)
        
        return data_ingestion_config

In [1]:
## Update Components
import os
import zipfile
from cnnclassifier.logging import logger
from kaggle.api.kaggle_api_extended import KaggleApi #download new token from kaggle for every session

In [43]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        self.api = KaggleApi()
        self.api.authenticate()

    def download_file(self):
        logger.info(f"Downloading Dataset: {self.config.dataset_name} to {self.config.local_data_file}")
        ## Extract the dataset 
        dataset_name = self.config.dataset_name
        output_path = self.config.local_data_file

        ## Download the dataset from kaggle
        self.api.dataset_download_files(dataset = dataset_name, path = output_path, unzip = True)
        logger.info(f"{dataset_name} extracted successfully to {output_path}")


In [45]:
## Create DataIngestionPipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    if "train_data.csv"  in os.listdir(data_ingestion_config.local_data_file):sys.exit()
    data_ingestion.download_file()
    source_dir = Path(os.path.join(data_ingestion_config.local_data_file, 
             os.listdir(data_ingestion_config.local_data_file)[0]))
    image_files = os.listdir(source_dir)
    for img in image_files:
        class_name = img.split(".")[0]
        ##create class directory if not exists
        class_directory = source_dir/class_name
        if not class_directory.exists():
            class_directory.mkdir()
        ## Move the image to class directory
        source_path = source_dir/img
        destination_path = class_directory/img
        shutil.move(src=str(source_path), dst=str(destination_path))
        
except Exception as e:
    raise e

[2024-08-26 16:30:11,308: INFO : common : config\config.yaml loaded successfully]
[2024-08-26 16:30:11,311: INFO : common : params\params.yaml loaded successfully]
[2024-08-26 16:30:11,314: INFO : common : secrets\secrets.yaml loaded successfully]
[2024-08-26 16:30:11,316: INFO : common : Parent Directories and Files Successfully Created]
[2024-08-26 16:30:11,318: INFO : common : Parent Directories and Files Successfully Created]


SystemExit: 